In [8]:
import cv2
import numpy as np

def cartoonify_image(image_path):
    # Read the image
    img = cv2.imread(image_path)
    
    # Check if the image was loaded correctly
    if img is None:
        print("Error: Could not load image. Please check the file path.")
        return
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply median blur
    gray = cv2.medianBlur(gray, 5)
    
    # Detect edges using adaptive thresholding
    edges = cv2.adaptiveThreshold(gray, 255, 
                                  cv2.ADAPTIVE_THRESH_MEAN_C, 
                                  cv2.THRESH_BINARY, 9, 9)
    
    # Apply bilateral filter to smooth the image while preserving edges
    color = cv2.bilateralFilter(img, 9, 300, 300)
    
    # Color quantization for color correction
    data = np.float32(color).reshape((-1, 3))
    k = 8  # Number of colors
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Convert back to uint8 and reshape
    centers = np.uint8(centers)
    quantized_color = centers[labels.flatten()]
    quantized_color = quantized_color.reshape(color.shape)
    
    # Combine edges and quantized color image
    cartoon = cv2.bitwise_and(quantized_color, quantized_color, mask=edges)
    
    # Display the original and cartoon images
    cv2.imshow("Original Image", img)
    cv2.imshow("Cartoon Image", cartoon)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
cartoonify_image("mymy.jpg")